# Limpieza y Preprocesamiento de Datos Financieros

Este notebook muestra paso a paso cómo funciona el módulo de limpieza de datos financieros.

**Objetivo**: Entender qué problemas existen en los datos crudos y cómo el `DataCleaner` los soluciona.

In [ ]:
import sys
from pathlib import Path

# Añadir src al path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from src.data import DataProvider
from src.data.cleaning import DataCleaner, CleaningMetadata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Módulos importados correctamente")

## 1. Carga de Datos "Sucios"

Primero, cargamos datos desde el DataProvider. Estos datos pueden tener problemas comunes:
- Valores nulos
- Duplicados
- Relaciones OHLC inválidas
- Outliers
- Problemas de formato

In [ ]:
# Inicializar proveedor de datos
provider = DataProvider()

# Obtener datos de precios
symbol = "AAPL"
raw_price_data = provider.get_price_data(symbol, period="1y", interval="1d")

print(f"Datos obtenidos para {symbol}")
print(f"Shape: {raw_price_data.shape}")
print(f"\nPrimeras filas:")
display(raw_price_data.head())
print(f"\nÚltimas filas:")
display(raw_price_data.tail())

## 2. Análisis Inicial de los Datos

Antes de limpiar, analicemos qué problemas tienen los datos.

In [ ]:
# Análisis de datos faltantes
print("=== ANÁLISIS DE VALORES FALTANTES ===")
missing = raw_price_data.isnull().sum()
missing_pct = (missing / len(raw_price_data)) * 100
missing_df = pd.DataFrame({
    'Valores Faltantes': missing,
    'Porcentaje': missing_pct
})
display(missing_df[missing_df['Valores Faltantes'] > 0])

# Análisis de duplicados
print(f"\n=== DUPLICADOS ===")
duplicates = raw_price_data.index.duplicated().sum()
print(f"Filas duplicadas: {duplicates}")

# Información general
print(f"\n=== INFORMACIÓN GENERAL ===")
print(f"Rango de fechas: {raw_price_data.index.min()} a {raw_price_data.index.max()}")
print(f"Total de días: {len(raw_price_data)}")
print(f"\nTipos de datos:")
print(raw_price_data.dtypes)

## 3. Visualización de Datos Antes de la Limpieza

Veamos cómo se ven los datos antes de limpiarlos.

In [ ]:
# Gráfico de precios antes de limpiar
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Precio de cierre
axes[0].plot(raw_price_data.index, raw_price_data['Close'], label='Close Price', linewidth=2)
axes[0].set_title(f'{symbol} - Precio de Cierre (Datos Crudos)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Fecha')
axes[0].set_ylabel('Precio (USD)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Volumen
axes[1].bar(raw_price_data.index, raw_price_data['Volume'], alpha=0.6, color='orange')
axes[1].set_title(f'{symbol} - Volumen (Datos Crudos)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Fecha')
axes[1].set_ylabel('Volumen')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Estadísticas descriptivas
print("\n=== ESTADÍSTICAS DESCRIPTIVAS (Datos Crudos) ===")
display(raw_price_data.describe())

## 4. Aplicación del DataCleaner

Ahora aplicamos el DataCleaner para limpiar los datos. Veamos qué hace paso a paso.

In [ ]:
# Crear instancia del limpiador
cleaner = DataCleaner(
    fill_method='forward',      # Forward-fill para valores nulos
    remove_outliers=False,      # No eliminar outliers automáticamente (solo detectar)
    validate_ohlc=True,         # Validar relaciones OHLC
    normalize_column_names=True # Normalizar nombres de columnas
)

# Limpiar datos
cleaned_price_data, metadata = cleaner.clean_price_data(raw_price_data, symbol=symbol)

print("=== LIMPIEZA COMPLETADA ===\n")
print(metadata.summary())

## 5. Comparación Antes vs Después

Comparemos los datos antes y después de la limpieza.

In [ ]:
# Comparación visual
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Precio de cierre - Antes
axes[0, 0].plot(raw_price_data.index, raw_price_data['Close'], label='Close', linewidth=2, color='red', alpha=0.7)
axes[0, 0].set_title('ANTES: Precio de Cierre', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Precio (USD)')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Precio de cierre - Después
axes[0, 1].plot(cleaned_price_data.index, cleaned_price_data['close'], label='Close', linewidth=2, color='green', alpha=0.7)
axes[0, 1].set_title('DESPUÉS: Precio de Cierre', fontsize=12, fontweight='bold')
axes[0, 1].set_ylabel('Precio (USD)')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Returns - Solo después (no existen antes)
if 'returns' in cleaned_price_data.columns:
    axes[1, 0].plot(cleaned_price_data.index, cleaned_price_data['returns'] * 100, linewidth=1, color='blue', alpha=0.6)
    axes[1, 0].set_title('DESPUÉS: Returns Diarios (%)', fontsize=12, fontweight='bold')
    axes[1, 0].set_ylabel('Returns (%)')
    axes[1, 0].axhline(y=0, color='black', linestyle='--', linewidth=1)
    axes[1, 0].grid(True, alpha=0.3)

# Distribución de returns
if 'returns' in cleaned_price_data.columns:
    axes[1, 1].hist(cleaned_price_data['returns'].dropna() * 100, bins=50, alpha=0.7, color='purple', edgecolor='black')
    axes[1, 1].set_title('DESPUÉS: Distribución de Returns', fontsize=12, fontweight='bold')
    axes[1, 1].set_xlabel('Returns (%)')
    axes[1, 1].set_ylabel('Frecuencia')
    axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Tabla comparativa
print("\n=== COMPARACIÓN DE ESTADÍSTICAS ===")
comparison = pd.DataFrame({
    'Antes (Close)': raw_price_data['Close'].describe(),
    'Después (close)': cleaned_price_data['close'].describe()
})
display(comparison)

## 6. Columnas Añadidas por el Limpiador

El DataCleaner añade columnas útiles para análisis.

In [ ]:
print("=== COLUMNAS ORIGINALES ===")
print(list(raw_price_data.columns))

print("\n=== COLUMNAS DESPUÉS DE LA LIMPIEZA ===")
print(list(cleaned_price_data.columns))

print("\n=== NUEVAS COLUMNAS AÑADIDAS ===")
new_columns = set(cleaned_price_data.columns) - set([col.lower() for col in raw_price_data.columns])
print(list(new_columns))

if new_columns:
    print("\n=== MUESTRA DE NUEVAS COLUMNAS ===")
    display(cleaned_price_data[list(new_columns)].head(10))

In [ ]:
# Obtener datos fundamentales
raw_fundamental = provider.get_fundamental_data(symbol)

print("=== DATOS FUNDAMENTALES CRUDOS (Primeros 10) ===")
fundamental_df = pd.DataFrame(list(raw_fundamental.items())[:10], columns=['Métrica', 'Valor'])
display(fundamental_df)

# Limpiar datos fundamentales
cleaned_fundamental, fundamental_metadata = cleaner.clean_fundamental_data(raw_fundamental)

print("\n=== METADATA DE LIMPIEZA ===")
print(fundamental_metadata.summary())

print("\n=== DATOS FUNDAMENTALES LIMPIOS (Primeros 10) ===")
cleaned_fundamental_df = pd.DataFrame(list(cleaned_fundamental.items())[:10], columns=['Métrica', 'Valor'])
display(cleaned_fundamental_df)

## 7. Limpieza de Estados Financieros

Finalmente, veamos cómo se limpian los estados financieros.

In [ ]:
# Obtener estados financieros
raw_income = provider.get_financial_statements(symbol, "income")

print("=== INCOME STATEMENT CRUDO (Primeras 10 filas) ===")
display(raw_income.head(10))

# Limpiar income statement
cleaned_income, income_metadata = cleaner.clean_financial_statement(raw_income, "income")

print("\n=== METADATA DE LIMPIEZA ===")
print(income_metadata.summary())

print("\n=== INCOME STATEMENT LIMPIO (Primeras 10 filas) ===")
display(cleaned_income.head(10))

## 8. Resumen y Lecciones Aprendidas

### ¿Qué problemas solucionó el DataCleaner?

1. **Normalización**: Nombres de columnas normalizados a minúsculas
2. **Ordenamiento**: Datos ordenados cronológicamente
3. **Duplicados**: Eliminación de filas duplicadas
4. **Validación OHLC**: Eliminación de relaciones inválidas (High < Low, etc.)
5. **Valores nulos**: Imputación inteligente según método elegido
6. **Tipos de datos**: Conversión a tipos numéricos correctos
7. **Columnas auxiliares**: Añadidas returns, log_returns, volatilidad, etc.
8. **Zona horaria**: Normalización de zonas horarias

### ¿Por qué es importante?

- **Análisis técnico**: Requiere datos limpios y consistentes
- **Backtesting**: Datos inválidos pueden causar resultados erróneos
- **Scoring**: Métricas calculadas sobre datos sucios son incorrectas
- **Visualización**: Gráficos más claros y precisos

### Próximos pasos

Los datos limpios están listos para:
- Análisis técnico (indicadores, estrategias)
- Análisis fundamental (ratios, métricas)
- Sistema de scoring
- Backtesting